In [128]:
### generate complex data 
import numpy as np
np.random.seed(5)

def generate_data(n_samples, true_coefficients, noise_level=0.1, poly_power=0.05):

    # Generate a random input signal
    k = len(true_coefficients[0])
    m, n = true_coefficients[0][0].shape
    p = len(true_coefficients[1])
    con = true_coefficients[0]
    w = true_coefficients[1]


    x = np.random.randn(n_samples, m) + 1j*np.random.randn(n_samples, m)

    # polynomial layer : w_1 * u + w_2 * |u|^2 * u  + w_3 * |u|^4 * u
    u = np.zeros((n_samples, m), dtype=np.complex128)
    for j in range(p):
        u += w[j] * np.power(np.abs(u), 2*j) * u * (poly_power ** j)
    


    # conv layer : convolution
    y = np.zeros((n_samples, n), dtype=np.complex128)
    for j in range(k):
        y += np.dot(np.roll(x, j, axis=0), con[j])



    # Add noise
    noise = noise_level * np.random.randn(n_samples, n) + 1j*noise_level * np.random.randn(n_samples, n)
    y += noise
    
    return x, y

# Example usage
n_samples = 300
# a series of matrix: k x m x n 
# k : influence scope 5
# m : output dimension  3
# n : input dimension 2
# p : polynomial degree 1, 3, 5, 3 coefficients in total
k = 5
m = 3
n = 2
p = 3

true_coefficients = [0,1 ]

### part 1 : polynomial layer
true_coefficients[1] = np.random.randn(p) + 1j*np.random.randn(p)

### part 2 : conv layer
true_coefficients[0] = [np.random.randn(m, n) + 1j*np.random.randn(m, n) for _ in range(k)]


noise_level = 0.05
poly_power = 0.05
x, y = generate_data(n_samples, true_coefficients, noise_level,poly_power)

print("Input signal:", x[:3])
print("Desired signal:", y[:3])

Input signal: [[ 1.189-1.707j -0.074+0.18j  -2.86 +3.612j]
 [ 0.789+0.295j -1.878-0.065j  1.539+0.484j]
 [ 1.821-0.034j -0.427+0.213j -1.165+0.272j]]
Desired signal: [[-3.837-6.449j -8.616-7.959j]
 [-2.177+3.033j  3.793-1.73j ]
 [ 4.322-4.423j  1.126+6.37j ]]


In [129]:
### LMS filter
## Hyperparameters
k = len(true_coefficients[0]) ## influence scope
p = len(true_coefficients[1]) ## polynomial degree
mu = 0.01

def compute_poly(x, m, p, poly_power_limit):
    """
    x : m x 1 complex vector
    PolyW : polynomial layer weights
    m : input dimension
    p : polynomial degree
    poly_power_limit : polynomial power limit

    return : a p x m complex vector
    """
    # out put : a nx1 complex vector

    # polynomial layer : w_1 * u + w_2 * |u|^2 * u  + w_3 * |u|^4 * u
    u = np.zeros((p, m), dtype=np.complex128)
    for j in range(p): 
        u[j] = np.power(np.abs(x), 2*j) * x * (poly_power_limit ** j)
    return u
def compute_poly_sum(u, PolyW, m, p):
    """
    u : p x m complex vector
    PolyW : polynomial layer weights
    m : input dimension
    p : polynomial degree

    return : a m x 1 complex vector
    """
    return np.dot(PolyW, u)

def compute_local_conv(x, ConvW, m, n, k, PolyW, p, poly_power_limit):
    """
    x : k x m complex vector
    ConvW : convolution layer weights
    m : input dimension
    n : output dimension
    k : influence scope
    """
    y = np.zeros((n), dtype=np.complex128)
    for j in range(k):
        poly = compute_poly(x[j], m, p, poly_power_limit)
        U = compute_poly_sum(poly, PolyW, m, p)
        
        y += np.dot(U, ConvW[j])
    return y

def lms_filter(x, y, k, mu, p):
    Ws = [] 
    m, n = x.shape[1], y.shape[1]

    PolyW = np.random.randn(p) + 1j*np.random.randn(p) ## p 
    ConvW = np.array([np.random.randn(m, n) + 1j*np.random.randn(m, n) for _ in range(k)]) ## k x m x n
    poly_power_limit = 0.05
    for i in range(k, len(x)):
        conv = compute_local_conv(x[i-k:i], ConvW, m, n, k, PolyW, p, poly_power_limit)
        y_hat = conv
        e =  y_hat - y[i]

        ### update weights
        # ConvW update
        for j in range(k):
            tmp = compute_poly(x[i-j], m, p, poly_power_limit)
            U = compute_poly_sum(tmp, PolyW, m, p)
            ConvW[j] += mu  * np.outer(np.conj(U),e)
        print(ConvW)
        for l in range(k):
            tmp = compute_poly(x[i-l], m, p, poly_power_limit)
            TMP =  np.dot(np.conj(ConvW[l]).T , np.conj(tmp).T)
            PolyW += mu/ k  *  np.dot(TMP.T,e)

        Ws.append((ConvW.copy(), PolyW.copy()))
    return  Ws


In [130]:
### Example usage
Ws = lms_filter(x, y, k, mu, p)

## print in 3 decimal places
np.set_printoptions(precision=3)

print("Estimated coefficients:", Ws[-1][0])
print("True coefficients:", true_coefficients[0])

# print("Estimated coefficients:", Ws[0][1])
# print("True coefficients:", true_coefficients[1])

[[[-0.481-0.676j -0.038+0.384j]
  [-0.612+3.253j  0.736-1.249j]
  [ 1.838+0.843j  0.341+0.953j]]

 [[-0.972+2.788j -1.843-0.711j]
  [ 0.149+0.531j  0.944-1.437j]
  [ 1.2  +1.406j  1.651+2.336j]]

 [[ 0.541-2.145j -0.501+1.034j]
  [ 0.769+2.24j  -0.121+0.69j ]
  [ 0.43 +0.485j  0.79 +0.01j ]]

 [[-1.12 +1.531j  0.085+1.069j]
  [ 2.142-1.97j   1.186-2.035j]
  [ 1.505-0.19j  -1.202-0.602j]]

 [[-1.221+1.41j   1.774-0.701j]
  [ 1.012+0.982j  0.847-0.754j]
  [-1.822+1.209j  0.837+1.211j]]]
[[[ 0.378 +4.681j -0.726 +3.388j]
  [-2.606 +5.549j -0.803 -0.464j]
  [ 0.526 -3.558j  0.587 -1.644j]]

 [[-1.289 +3.839j -2.234 -0.225j]
  [ 6.51 +10.971j  2.07  +5.413j]
  [ 2.189 +5.741j  1.249 +4.829j]]

 [[ 4.314 -0.75j   1.189 +2.572j]
  [ 1.829 +2.362j  0.411 +0.98j ]
  [-1.79  +8.046j -1.988 +3.519j]]

 [[ 0.097 -3.234j  1.74  -1.181j]
  [ 2.993 +2.658j  0.649 +0.583j]
  [ 2.565 +0.095j -0.705 -0.227j]]

 [[-5.297 +5.179j -1.174 +0.416j]
  [ 2.276 +0.579j  1.598 -0.698j]
  [ 1.136 -0.558j  2.771 +

In [131]:
true_coefficients[1]

array([ 0.441-0.252j, -0.331+0.11j ,  2.431+1.582j])